In [1]:
import pandas as pd
import numpy as np
import os

from pyproj import Transformer
from pathlib import Path

In [2]:
people_df = pd.read_csv("../data/persons_home_depot_with_start_duration_draws.csv")
all_activities_df = pd.read_csv("../data/activities_long_with_groups_selected_plus.csv")
charger_df = pd.read_csv("../data/charger_location_attributes_dropped_final.csv")

/var/folders/p0/bcd_l4996lz24gnzf87fwjm80000gn/T/ipykernel_49553/1543247342.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  people_df = pd.read_csv("../data/persons_home_depot_with_start_duration_draws.csv")


In [3]:
# pid_to_select = "2002000535_5_E02001617_1_2002001259"
pid_to_select = "2002000537_3_E02001571_1_2002001263"
pid_identifier = pid_to_select[-4:]

group_to_type = {
        1: "home",
        2: "work",
        3: "business",
        4: "shop/visit",
        5: "education",
        6: "depot/medical",
        7: "delivery/visit",
        8: "other/escort",
    }


In [4]:
activities_df = all_activities_df[all_activities_df["pid"] == pid_to_select].drop('pid', axis = 1)
activities_df["act_type"]=activities_df["group"].map(group_to_type)

In [5]:
newpath = f"person_ending_{pid_identifier}/"
if not os.path.exists(newpath):
    os.makedirs(newpath)

activities_df.to_csv(f"{newpath}activities.csv", index=False)

In [6]:
individual_df = people_df[people_df["pid"]==pid_to_select]

In [7]:
def create_test_individual_csv(person, output_dir):
    """Create a simplified individual CSV for testing."""
    # Mapping from DP group numbers to activity type names
    group_to_type = {
        1: "home",
        2: "work",
        3: "business",
        4: "shop/visit",
        5: "education",
        6: "depot/medical",
        7: "delivery/visit",
        8: "other/escort",
    }

    individual_data = {
        "pid": person["pid"].values[0],
        "home_x": person["x"].values[0],
        "home_y": person["y"].values[0],
        # "work_x": person["x"].values[0],  # Simplified: use home as work
        # "work_y": person["y"].values[0],
        "age_group": person["age_group"].values[0],
        "workstatus": person["workstatus"].values[0],
    }

    # Add start times and durations for each group
    for group in range(1, 9):
        if group in group_to_type:
            activity_type = group_to_type[group]

            start_col = f"start_g{group}"
            dur_col = f"dur_g{group}"

            if start_col in person.columns:
                individual_data[f"{activity_type}_start"] = (
                    person[start_col].values[0]
                    if not pd.isna(person[start_col].values[0])
                    else 0
                )
            else:
                individual_data[f"{activity_type}_start"] = 0

            if dur_col in person.columns:
                individual_data[f"{activity_type}_dur"] = (
                    person[dur_col].values[0]
                    if not pd.isna(person[dur_col].values[0])
                    else 12
                )
            else:
                individual_data[f"{activity_type}_dur"] = 12

    individual_df = pd.DataFrame([individual_data])
    output_path = output_dir + "individual_metadata.csv"
    individual_df.to_csv(output_path, index=False)

    print(f"\nSaved individual data to {output_path}")

    return individual_df

In [8]:
individual_df = create_test_individual_csv(individual_df, newpath)


Saved individual data to person_ending_1263/individual_metadata.csv


In [9]:
individual_df.columns

Index(['pid', 'home_x', 'home_y', 'age_group', 'workstatus', 'home_start',
       'home_dur', 'work_start', 'work_dur', 'business_start', 'business_dur',
       'shop/visit_start', 'shop/visit_dur', 'education_start',
       'education_dur', 'depot/medical_start', 'depot/medical_dur',
       'delivery/visit_start', 'delivery/visit_dur', 'other/escort_start',
       'other/escort_dur'],
      dtype='object')

In [10]:
activities_df

,activity_idx,act_type,x,y,group,earliest_start,latest_start,max_duration,min_duration
82,1,home,451322.0,396724.0,1,0,288,288,2
83,2,work,446986.0,399916.0,2,60,276,144,2
84,3,other/escort,449714.0,399146.0,8,108,216,120,2
85,4,home,451322.0,396724.0,1,0,288,288,2
86,5,other/escort,457303.0,403122.0,8,108,216,120,2
87,6,home,451322.0,396724.0,1,0,288,288,2
88,7,depot/medical,450688.0,399064.0,6,84,240,96,2
89,8,depot/medical,451235.0,393088.0,6,84,240,96,2
90,9,depot/medical,455564.0,399279.0,6,84,240,96,2
91,10,depot/medical,457942.0,392624.0,6,84,240,96,2


In [11]:
start_times = {act_type: int(individual_df[f"{act_type}_start"].values[0]) 
               for act_type in group_to_type.values() 
               if f"{act_type}_start" in individual_df.columns}

durations = {act_type: int(individual_df[f"{act_type}_dur"].values[0]) 
             for act_type in group_to_type.values() 
             if f"{act_type}_dur" in individual_df.columns}

In [12]:
start_times

{'home': 0,
 'work': 95,
 'business': 100,
 'shop/visit': 180,
 'education': 151,
 'depot/medical': 0,
 'delivery/visit': 0,
 'other/escort': 109}

In [13]:
# Map des_start_time and des_duration from individual preferences
activities_df["des_start_time"] = activities_df["act_type"].map(start_times).fillna(0).astype(int)
activities_df["des_duration"] = activities_df["act_type"].map(durations).fillna(0).astype(int)

In [14]:
activities_df["act_type"].values

array(['home', 'work', 'other/escort', 'home', 'other/escort', 'home',
       'depot/medical', 'depot/medical', 'depot/medical', 'depot/medical',
       'depot/medical'], dtype=object)

In [15]:
# Get home coordinates from individual
home_x = individual_df["home_x"].values[0]
home_y = individual_df["home_y"].values[0]

HORIZON = 288  # Assuming 5-minute intervals over 24 hours (288 intervals)

# Create dawn activity (index 0) - must start at time 0
dawn_activity = pd.DataFrame([{
    "id": 0,
    "act_type": "home",
    "x": home_x,
    "y": home_y,
    "group": 1,  # home group
    "earliest_start": 0,
    "latest_start": 0,
    "max_duration": HORIZON - 2,
    "min_duration": 1,
    "des_start_time": 0,
    "des_duration": 0
}])

# Prepare the middle activities (from the original activities_df)
# Reset index and create proper id column starting from 1
middle_activities = activities_df.copy()
middle_activities = middle_activities.reset_index(drop=True)
middle_activities["id"] = middle_activities.index + 1  # IDs start from 1

# Select only the columns needed (now including act_type)
middle_activities = middle_activities[["id", "act_type", "x", "y", "group", "earliest_start", 
                                        "latest_start", "max_duration", "min_duration",
                                        "des_start_time", "des_duration"]]

# Calculate the next id for the special activities at the end
next_id = len(middle_activities) + 1

# Create the second-to-last home activity (with des_duration=0, des_start_time=0)
home_activity_special = pd.DataFrame([{
    "id": next_id,
    "act_type": "home",
    "x": home_x,
    "y": home_y,
    "group": 1,  # home group
    "earliest_start": 0,
    "latest_start": HORIZON,
    "max_duration": HORIZON - 2,
    "min_duration": 1,
    "des_start_time": 0,
    "des_duration": 0
}])

# Create dusk activity (last index) - can end anytime
dusk_activity = pd.DataFrame([{
    "id": next_id + 1,
    "act_type": "home",
    "x": home_x,
    "y": home_y,
    "group": 1,  # home group
    "earliest_start": 0,
    "latest_start": HORIZON - 2,
    "max_duration": HORIZON - 2,
    "min_duration": 1,
    "des_start_time": 0,
    "des_duration": 0
}])

# Concatenate all activities in order
final_activities_df = pd.concat([dawn_activity, middle_activities, home_activity_special, dusk_activity], 
                                 ignore_index=True)

print(f"Total activities: {len(final_activities_df)}")
print(f"Dawn (id=0), Middle activities (id=1 to {next_id-1}), Special home (id={next_id}), Dusk (id={next_id+1})")
final_activities_df

Total activities: 14
Dawn (id=0), Middle activities (id=1 to 11), Special home (id=12), Dusk (id=13)


,id,act_type,x,y,group,earliest_start,latest_start,max_duration,min_duration,des_start_time,des_duration
0,0,home,451322.0,396724.0,1,0,0,286,1,0,0
1,1,home,451322.0,396724.0,1,0,288,288,2,0,82
2,2,work,446986.0,399916.0,2,60,276,144,2,95,146
3,3,other/escort,449714.0,399146.0,8,108,216,120,2,109,4
4,4,home,451322.0,396724.0,1,0,288,288,2,0,82
5,5,other/escort,457303.0,403122.0,8,108,216,120,2,109,4
6,6,home,451322.0,396724.0,1,0,288,288,2,0,82
7,7,depot/medical,450688.0,399064.0,6,84,240,96,2,0,12
8,8,depot/medical,451235.0,393088.0,6,84,240,96,2,0,12
9,9,depot/medical,455564.0,399279.0,6,84,240,96,2,0,12


In [16]:
# charge_mode,is_charging,is_service_station,power_range_max

final_activities_df["charge_mode"]=0
final_activities_df["is_charging"]=0
final_activities_df["is_service_station"]=0

In [17]:
final_activities_df

,id,act_type,x,y,group,earliest_start,latest_start,max_duration,min_duration,des_start_time,des_duration,charge_mode,is_charging,is_service_station
0,0,home,451322.0,396724.0,1,0,0,286,1,0,0,0,0,0
1,1,home,451322.0,396724.0,1,0,288,288,2,0,82,0,0,0
2,2,work,446986.0,399916.0,2,60,276,144,2,95,146,0,0,0
3,3,other/escort,449714.0,399146.0,8,108,216,120,2,109,4,0,0,0
4,4,home,451322.0,396724.0,1,0,288,288,2,0,82,0,0,0
5,5,other/escort,457303.0,403122.0,8,108,216,120,2,109,4,0,0,0
6,6,home,451322.0,396724.0,1,0,288,288,2,0,82,0,0,0
7,7,depot/medical,450688.0,399064.0,6,84,240,96,2,0,12,0,0,0
8,8,depot/medical,451235.0,393088.0,6,84,240,96,2,0,12,0,0,0
9,9,depot/medical,455564.0,399279.0,6,84,240,96,2,0,12,0,0,0


In [18]:
# Save the final activities dataframe
output_path = f"{newpath}activities_with_charge.csv"
final_activities_df.to_csv(output_path, index=False)
print(f"Saved activities to {output_path}")

Saved activities to person_ending_1263/activities_with_charge.csv
